# Local Electronic Structure and Dynamics of Muon-Polaron Complexes in Fe$_2$O$_3$

## Supplemental material: Separating the muon-polaron complexes

M. H. Dehn$^{1, 2, 3}$ J. K. Shenton$^{4,*}$ D. J. Arseneau$^3$ W. A. MacFarlane$^{2, 3, 5}$ G.
D. Morris$^3$ A. Maigné$^2$ N. A. Spaldin$^4$, and R. F. Kiefl$^{1, 2, 3}$


$^1$Department of Physics and Astronomy, University of British Columbia, Vancouver, BC V6T 1Z1, Canada    
$^2$Stewart Blusson Quantum Matter Institute, University of British Columbia, Vancouver, BC V6T 1Z4, Canada    
$^3$<span style="font-variant:small-caps;">Triumf</span>, Vancouver, BC V6T 2A3, Canada   
$^4$Department of Materials, ETH Zurich, CH-8093 Zürich, Switzerland   
$^5$Department of Chemistry, University of British Columbia, Vancouver, BC, V6T 1Z1, Canada    




$^*$ For queries about the supplemental information in this notebook contact [J. Kane Shenton](mailto:john.shenton@mat.ethz.ch).

### Summary

In this notebook we explore the energetics of separating muon-polaron complexes. To do this we require a significantly larger supercell than elsewhere in this study. The following are results for a hexagonal 3x3x1 supercell (270+µ atoms).

The results below suggest the following:
1. C$^0_1$ is lower in energy than C$^+$ and a separated polaron.
2. C$^-_1$ is lower in energy than C$^0_1$ and a separated polaron.


Given the long-range nature of the electrostatic interaction between these defects, we propose further investigation at either larger supercell sizes, and/or appropriate correction schemes.

### Procedure

We are interested in comparing the energies of systems as a function of muon$-$polaron separation. Due to the distortions generated by the muon and its positive charge, excess charge spontaneously localises close to the muon in the course of electronic and ionic relaxations using VASP. In order, therefore, to separate the muon from the polaron we start from the relaxed muon-polaron complex (C$^0_1$ in the paper) and translate the muon to symmetry-similar sites in the supercell and re-relaxed the ionic positions. In certain cases the polaron ended up at the new muon position (therefore identical to the configurations in the paper), but in general this procedure successfully separated the muon from the polaron. Thus we compare the energies of the muon-polaron complex with those where the polaron is separated from the muon in the unit cell.

Starting from the relaxed separated structures above, we then add in a second 'excess' electron to the supercell (i.e. have an overall supercell charge of -1) and re-relax. We find cases in which one polaron localises near the new muon position whilst the other remains further away. Thus we can thus compare the energies of a muon+two polaron complex (C$^-_1$ in the paper) to a supercell with a charge-neutral muon-polaron complex and a second polaron further away.


In order to estimate the position of the polaron(s), we look at the local magnetic moments on the Fe ions. Fe$^{3+}$ in an octahedral environment in the high-spin state is $d^5$ with a nominal spin of 5 $\mu_B$. Due to hybridisation with O and the limitations of the projection scheme used to calculate the local moments ([LORBIT=11](https://www.vasp.at/wiki/index.php/LORBIT)), the local spin moments are found to be ~4 $\mu_B$.$^*$ The localisation of a polaron on a given Fe site then results in a _decrease_ of this local moment to between 3-4 µB, depending on the degree of localisation. We generally find that, with the LDA + U = 4 eV setup used here, a polaron will predominantly localise on one Fe ion, with some localisation on a neighbouring Fe ion. 


$^*$ The actual value depends on the choice of Hubbard 'U' and the projection scheme, amongst other factors. 


### Computational details

* Supercell size: 3x3x1 
    * 270 atoms + 1 'muon'
    * a, b, c = 14.880518787, 14.880518787, 13.595788232 Å
    * $\alpha$, $\beta$, $\gamma$ = 90, 90, 120 $^\circ$
* DFT code: VASP version 5.4.4
* Plane-wave cutoff energy: 700 eV
* $\Gamma$-point only k-point sampling
* Exchange-correlation functional: LDA
* $\mathrm{U_{eff}} = 4 $ eV Hubbard correction (Dudarev scheme)
* SCF energy tolerance: 1E-6 eV
* Maximum force tolerance: 5 meV/Å
* The following PAWs were used:
    * Fe_pv 02Aug2007
    * O 22Mar2012
    * H 06May1998

The full `INCAR`, `KPOINTS`, `POSCAR`, `CONTCAR` and `OUTCAR` files can be found in the subdirectories of `./convergence_tests/separate_muon-polaron/`

To set up, run and parse the results we make extensive use of [ASE](https://wiki.fysik.dtu.dk/ase/index.html).

### Setup

We import standard packages to be used in the analysis.

In [1]:
# ASE version 3.19.1
from ase.io import read

# re version: 2.2.1
import re

import glob

In order to reduce the size of this repository, we have compressed all of the files used for this analysis. In order to compress or decompress the files, run one of the cells below:

In [57]:
# --- Compress --- #
!gzip -r convergence_tests/separate_muon-polaron/

In [ ]:
# --- De-compress --- #
!gzip -dr convergence_tests/separate_muon-polaron/

### Separating the neutral muon-polaron complex

The sites are the rough-relaxed hexagonal 331 supercell of the C$^0_1$ case but with the muon translated by:


The vec_a sites are initialised as following:

vec_a = [2,4,1]/6   
(Defined with respect to the 30-atom primitive hexagonal unit cell)

    site_1: vec_a*1
    site_2: vec_a*2
    site_3: vec_a*3
    
    
    
The vec_ab sites are initialised as following:


vec_a = [2,4,1]/6   
vec_b = [4,2,-1]/6   

    site_1: vec_a + vec_b
    site_2: vec_a*2 + vec_b
    site_3: vec_a*2 + vec_b*2


The starting strcuture is the hexagonal 331 supercell with the C$^0_1$ site as found here:

In [2]:
basedir = "./convergence_tests/separate_muon-polaron/mu_zero/"
muzero_C1 = read(basedir + "hex331_muzero_1/20200417-2139-OUTCAR", format='vasp-out', index=-1)

#### Translating along vec a

In [48]:
# Lists to store the structures and local magnetic moments
vec_a_structures = []
vec_a_Fe_mags = []
vec_a_order = []
vec_a_distances = []

# Gather the time-stamped filenames:
sitefnames = glob.glob('./convergence_tests/separate_muon-polaron/mu_zero/vec_a/site*/relax-1/2020*OUTCAR')
sitefnames.sort() # to get the sites in order

# Loop over sites and read in the results:
for site in sitefnames:
    basedir = '/'.join(site.split('/')[:-2])
    site_index = basedir[-1]
    print(site)
    
    atoms = read(site, format='vasp-out', index=-1)
    mags = atoms.get_magnetic_moments()
    vec_a_structures.append(atoms)
    
    Fe_indices = [atom.index for atom in atoms if atom.symbol == 'Fe']

    # Sort the Fe indices by abs. magnetic moment
    Fe_mag_order = np.argsort(np.abs(mags[Fe_indices]))
    vec_a_order.append(Fe_mag_order)

    # Fe mag. moms sorted by their absoulte value    
    Fe_mags = mags[Fe_indices][Fe_mag_order]
    vec_a_Fe_mags.append(Fe_mags)

    # Distances from muon to Fe ion:    
    # We use the abs. mag. moments on the Fe ions to sort them
    # (the muon index is -1)
    distances = atoms.get_distances(-1, Fe_mag_order, mic=True)
    vec_a_distances.append(distances)
    
# Convert to numpy arrays
vec_a_Fe_mags = np.array(vec_a_Fe_mags)
vec_a_distances = np.array(vec_a_distances)
vec_a_order = np.array(vec_a_order)

for site in range(len(sitefnames)):
    print(f"\nSite {site+1}: 5 lowest local moments")
    print('Fe index:  {0: 6d} {1: 6d} {2: 6d} {3: 6d} {4: 6d}'.format(*vec_a_order[site,0:5]))
    print('Mag. mom. : {0:6.3f} {1:6.3f} {2:6.3f} {3:6.3f} {4:6.3f} µB'.format(*vec_a_Fe_mags[site,0:5]))
    print('µ-Fe dist.: {0:6.3f} {1:6.3f} {2:6.3f} {3:6.3f} {4:6.3f} Å'.format(*vec_a_distances[site,0:5]))

print("\nFor reference, the mean of 10 the largest local moments is")
print(f"mags: {np.abs(vec_a_Fe_mags)[:,-10:].mean():6.3f} µB")

./convergence_tests/separate_muon-polaron/mu_zero/vec_a/site-1/relax-1/20200419-0146-OUTCAR
./convergence_tests/separate_muon-polaron/mu_zero/vec_a/site-2/relax-1/20200419-0210-OUTCAR
./convergence_tests/separate_muon-polaron/mu_zero/vec_a/site-3/relax-1/20200419-0209-OUTCAR

Site 1: 5 lowest local moments
Fe index:      11     22     44     47      5
Mag. mom. : -3.673 -3.892  3.998 -4.013  4.016 µB
µ-Fe dist.:  3.864  4.724  2.068  5.484  4.731 Å

Site 2: 5 lowest local moments
Fe index:      39     98     36     35     99
Mag. mom. : -3.680 -3.873  4.016 -4.017 -4.017 µB
µ-Fe dist.:  2.106  4.688  4.826  4.809  6.518 Å

Site 3: 5 lowest local moments
Fe index:      11     22     84     93     82
Mag. mom. : -3.765 -3.814  4.000  4.015 -4.015 µB
µ-Fe dist.:  9.895  9.968  2.080  4.731  9.177 Å

For reference, the mean of 10 the largest local moments is
mags:  4.028 µB


We see that the polaron remains localised on the two Fe ions: 11 and 22 for sites 1 and 3, but for site 2, the polaron  jumped to Fe ions 39 and 98. We shall see below that this is close to the muon. 

#### Translating along vec ab

In [49]:
# Lists to store the structures and local magnetic moments
vec_ab_structures = []
vec_ab_Fe_mags = []
vec_ab_order = []
vec_ab_distances = []

# Gather the time-stamped filenames:
sitefnames = glob.glob('./convergence_tests/separate_muon-polaron/mu_zero/vec_ab/site*/relax-1/2020*OUTCAR')
sitefnames.sort() # to get the sites in order

# Loop over sites and read in the results:
for site in sitefnames:
    basedir = '/'.join(site.split('/')[:-2])
    site_index = basedir[-1]
    print(site)
    
    atoms = read(site, format='vasp-out', index=-1)
    mags = atoms.get_magnetic_moments()
    vec_ab_structures.append(atoms)
    
    Fe_indices = [atom.index for atom in atoms if atom.symbol == 'Fe']

    # Sort the Fe indices by abs. magnetic moment
    Fe_mag_order = np.argsort(np.abs(mags[Fe_indices]))
    vec_ab_order.append(Fe_mag_order)

    # Fe mag. moms sorted by their absoulte value    
    Fe_mags = mags[Fe_indices][Fe_mag_order]
    vec_ab_Fe_mags.append(Fe_mags)

    # Distances from muon to Fe ion:    
    # We use the abs. mag. moments on the Fe ions to sort them
    # (the muon index is -1)
    distances = atoms.get_distances(-1, Fe_mag_order, mic=True)
    vec_ab_distances.append(distances)
    
# Convert to numpy arrays
vec_ab_Fe_mags = np.array(vec_ab_Fe_mags)
vec_ab_distances = np.array(vec_ab_distances)
vec_ab_order = np.array(vec_ab_order)

for site in range(len(sitefnames)):
    print(f"\nSite {site+1}: 5 lowest local moments")
    print('Fe index:  {0: 6d} {1: 6d} {2: 6d} {3: 6d} {4: 6d}'.format(*vec_ab_order[site,0:5]))
    print('Mag. mom. : {0:6.3f} {1:6.3f} {2:6.3f} {3:6.3f} {4:6.3f} µB'.format(*vec_ab_Fe_mags[site,0:5]))
    print('µ-Fe dist.: {0:6.3f} {1:6.3f} {2:6.3f} {3:6.3f} {4:6.3f} Å'.format(*vec_ab_distances[site,0:5]))

print("\nFor reference, the mean of 10 the largest local moments is")
print(f"mags: {np.abs(vec_ab_Fe_mags)[:,-10:].mean():6.3f} µB")

./convergence_tests/separate_muon-polaron/mu_zero/vec_ab/site-1/relax-1/20200421-1601-OUTCAR
./convergence_tests/separate_muon-polaron/mu_zero/vec_ab/site-2/relax-1/20200420-0157-OUTCAR
./convergence_tests/separate_muon-polaron/mu_zero/vec_ab/site-3/relax-1/20200420-1102-OUTCAR

Site 1: 5 lowest local moments
Fe index:      22     11     46     43     69
Mag. mom. : -3.671 -3.895 -3.999 -4.015  4.016 µB
µ-Fe dist.:  3.395  3.872  2.087  4.726  5.202 Å

Site 2: 5 lowest local moments
Fe index:      22     11    104     47     82
Mag. mom. : -3.696 -3.876  3.998 -4.014 -4.015 µB
µ-Fe dist.:  5.687  6.969  2.071  3.896  7.883 Å

Site 3: 5 lowest local moments
Fe index:      11     22    106     47     82
Mag. mom. : -3.704 -3.868 -3.998 -4.015 -4.015 µB
µ-Fe dist.:  6.784  6.846  2.078  3.952  5.201 Å

For reference, the mean of 10 the largest local moments is
mags:  4.028 µB


We see that the polaron remains localised on the two Fe ions: 22 and 11 for each site, despite the muon moving away. 

#### Summary

In [50]:
# For reference, let us print out the properties of the unseparated µ-polaron complex in C1:
muzero_C1_energy = muzero_C1.get_potential_energy() # eV
mags = abs(muzero_C1.get_magnetic_moments())
muzero_C1_polaron_order =np.argsort(mags[:108])
dist1 = muzero_C1.get_distance(-1, muzero_C1_polaron_order[0], mic=True)
dist2 = muzero_C1.get_distance(-1, muzero_C1_polaron_order[1], mic=True)
print(f"Site: 0, \
Dist µ-pol: {dist1:5.2f} Å, \
E wrt C1: {0:5.1f} meV")


print("\nVec a")
for i, atoms in enumerate(vec_a_structures):
    dist1 = atoms.get_distance(-1, vec_a_order[:,0][i], mic=True)
    dist2 = atoms.get_distance(-1, vec_a_order[:,1][i], mic=True)
    e = atoms.get_potential_energy()
    print(f"Site: {i+1}, \
Dist µ-pol: {dist1:5.2f} Å, \
E wrt C1: {(e - muzero_C1_energy)*1000:5.1f} meV ")
    
print("\nVec ab")
for i, atoms in enumerate(vec_ab_structures):
    dist1 = atoms.get_distance(-1, vec_ab_order[:,0][i], mic=True)
    dist2 = atoms.get_distance(-1, vec_ab_order[:,1][i], mic=True)
    e = atoms.get_potential_energy()
    print(f"Site: {i+1}, \
Dist µ-pol: {dist1:5.2f} Å, \
E wrt C1: {(e - muzero_C1_energy)*1000:5.1f} meV ")

Site: 0, Dist µ-pol:  2.11 Å, E wrt C1:   0.0 meV

Vec a
Site: 1, Dist µ-pol:  3.86 Å, E wrt C1: 157.9 meV 
Site: 2, Dist µ-pol:  2.11 Å, E wrt C1:   0.0 meV 
Site: 3, Dist µ-pol:  9.90 Å, E wrt C1: 201.7 meV 

Vec ab
Site: 1, Dist µ-pol:  3.39 Å, E wrt C1: 159.8 meV 
Site: 2, Dist µ-pol:  5.69 Å, E wrt C1: 177.5 meV 
Site: 3, Dist µ-pol:  6.78 Å, E wrt C1: 174.9 meV 


For Vec a:   
In the case of Site 2, the polaron re-localised right next to the new muon position in a similar configuration to the initial Site 0 (i.e. C$^0_1$) and thus had almost identical properties.

Sites 1 and 3, however, did successfully trap the polaron further away from the muon resulting in a significant increase in total energy. 


For Vec ab:   
Sites 1,2 and 3 successfully trapped the polaron further away from the muon, resulting in a significant increase in total energy.


We can therefore say that the muon-polaron complex is more stable than the configurations tested for the separated muon and polaron. 


**Overall we generally see an increase in energy as a function of muon-polaron separation.**

### Separating the muon-polaron complex from a second polaron (charge -1)

We start from the relaxed charge-neutral sites found above and add an extra electron into the system to try to for a muon-polaron complex + another polaron further away. 

The reference system for the following is the C$^-_1$ configuration in the hexagonal 3x3x1 cell:

In [51]:
basedir = "./convergence_tests/separate_muon-polaron/mu_minus/"
muminus_C1 = read(basedir + "hex331_muminus_1/20200418-2246-OUTCAR", format='vasp-out', index=-1)

#### Translating along vec a

In [52]:
# Lists to store the structures and local magnetic moments
vec_a_structures = []
vec_a_Fe_mags = []
vec_a_order = []
vec_a_distances = []

# Gather the time-stamped filenames:
sitefnames = glob.glob('./convergence_tests/separate_muon-polaron/mu_minus/vec_a/site*/relax/2020*OUTCAR')
sitefnames.sort() # to get the sites in order

# Loop over sites and read in the results:
for site in sitefnames:
    basedir = '/'.join(site.split('/')[:-2])
    site_index = basedir[-1]
    print(site)
    
    atoms = read(site, format='vasp-out', index=-1)
    mags = atoms.get_magnetic_moments()
    vec_a_structures.append(atoms)
    
    Fe_indices = [atom.index for atom in atoms if atom.symbol == 'Fe']

    # Sort the Fe indices by abs. magnetic moment
    Fe_mag_order = np.argsort(np.abs(mags[Fe_indices]))
    vec_a_order.append(Fe_mag_order)

    # Fe mag. moms sorted by their absoulte value    
    Fe_mags = mags[Fe_indices][Fe_mag_order]
    vec_a_Fe_mags.append(Fe_mags)

    # Distances from muon to Fe ion:    
    # We use the abs. mag. moments on the Fe ions to sort them
    # (the muon index is -1)
    distances = atoms.get_distances(-1, Fe_mag_order, mic=True)
    vec_a_distances.append(distances)
    
# Convert to numpy arrays
vec_a_Fe_mags = np.array(vec_a_Fe_mags)
vec_a_distances = np.array(vec_a_distances)
vec_a_order = np.array(vec_a_order)

for site in range(len(sitefnames)):
    print(f"\nSite {site+1}: 5 lowest local moments")
    print('Fe index:  {0: 6d} {1: 6d} {2: 6d} {3: 6d} {4: 6d}'.format(*vec_a_order[site,0:5]))
    print('Mag. mom. : {0:6.3f} {1:6.3f} {2:6.3f} {3:6.3f} {4:6.3f} µB'.format(*vec_a_Fe_mags[site,0:5]))
    print('µ-Fe dist.: {0:6.3f} {1:6.3f} {2:6.3f} {3:6.3f} {4:6.3f} Å'.format(*vec_a_distances[site,0:5]))

print("\nFor reference, the mean of 10 the largest local moments is")
print(f"mags: {np.abs(vec_a_Fe_mags)[:,-10:].mean():6.3f} µB")

./convergence_tests/separate_muon-polaron/mu_minus/vec_a/site-1/relax/20200421-1226-OUTCAR
./convergence_tests/separate_muon-polaron/mu_minus/vec_a/site-2/relax/20200422-1620-OUTCAR
./convergence_tests/separate_muon-polaron/mu_minus/vec_a/site-3/relax/20200420-2033-OUTCAR

Site 1: 5 lowest local moments
Fe index:      11     44     37     22      8
Mag. mom. : -3.607  3.767  3.791 -3.937  4.013 µB
µ-Fe dist.:  3.808  2.055  4.543  4.665  3.669 Å

Site 2: 5 lowest local moments
Fe index:      25     39     98     32     86
Mag. mom. :  3.617 -3.645 -3.914  3.918 -4.009 µB
µ-Fe dist.:  2.129  2.469  5.101  4.148  2.128 Å

Site 3: 5 lowest local moments
Fe index:      84     11     22    101     93
Mag. mom. :  3.699 -3.782 -3.795  3.853  4.014 µB
µ-Fe dist.:  2.057  9.948  9.982  4.561  4.758 Å

For reference, the mean of 10 the largest local moments is
mags:  4.027 µB


For Site 1: one polaron lives on Fe ions 11 and 22 and the other on Fe ions 44 and 37.   
For Site 2: one polaron lives on predominantly on Fe ion 25 and the other predominantly on Fe ion 39.   
For Site 3: one polaron lives on Fe ions 11 and 22 and the other on Fe ions 84 and 101.   

#### Translating along vec ab

In [53]:
# Lists to store the structures and local magnetic moments
vec_ab_structures = []
vec_ab_Fe_mags = []
vec_ab_order = []
vec_ab_distances = []
vec_ab_site_inds = []

# Gather the time-stamped filenames:
sitefnames = glob.glob('./convergence_tests/separate_muon-polaron/mu_minus/vec_ab/site*/relax/2020*OUTCAR')
sitefnames.sort() # to get the sites in order

# Loop over sites and read in the results:
for site in sitefnames:
    basedir = '/'.join(site.split('/')[:-2])
    site_index = basedir[-1]
    print(site)
    vec_ab_site_inds.append(site_index)

    
    atoms = read(site, format='vasp-out', index=-1)
    mags = atoms.get_magnetic_moments()
    vec_ab_structures.append(atoms)
    
    Fe_indices = [atom.index for atom in atoms if atom.symbol == 'Fe']

    # Sort the Fe indices by abs. magnetic moment
    Fe_mag_order = np.argsort(np.abs(mags[Fe_indices]))
    vec_ab_order.append(Fe_mag_order)

    # Fe mag. moms sorted by their absoulte value    
    Fe_mags = mags[Fe_indices][Fe_mag_order]
    vec_ab_Fe_mags.append(Fe_mags)

    # Distances from muon to Fe ion:    
    # We use the abs. mag. moments on the Fe ions to sort them
    # (the muon index is -1)
    distances = atoms.get_distances(-1, Fe_mag_order, mic=True)
    vec_ab_distances.append(distances)
    
# Convert to numpy arrays
vec_ab_Fe_mags = np.array(vec_ab_Fe_mags)
vec_ab_distances = np.array(vec_ab_distances)
vec_ab_order = np.array(vec_ab_order)

for isite, site in enumerate(vec_ab_site_inds):
    print(f"\nSite {site}: 5 lowest local moments")
    print('Fe index:  {0: 6d} {1: 6d} {2: 6d} {3: 6d} {4: 6d}'.format(*vec_ab_order[isite,0:5]))
    print('Mag. mom. : {0:6.3f} {1:6.3f} {2:6.3f} {3:6.3f} {4:6.3f} µB'.format(*vec_ab_Fe_mags[isite,0:5]))
    print('µ-Fe dist.: {0:6.3f} {1:6.3f} {2:6.3f} {3:6.3f} {4:6.3f} Å'.format(*vec_ab_distances[isite,0:5]))

print("\nFor reference, the mean of 10 the largest local moments is")
print(f"mags: {np.abs(vec_ab_Fe_mags)[:,-10:].mean():6.3f} µB")

./convergence_tests/separate_muon-polaron/mu_minus/vec_ab/site-1/relax/20200420-2052-OUTCAR
./convergence_tests/separate_muon-polaron/mu_minus/vec_ab/site-3/relax/20200421-1129-OUTCAR

Site 1: 5 lowest local moments
Fe index:      22     46     95     11     56
Mag. mom. : -3.659 -3.710 -3.842 -3.899  4.012 µB
µ-Fe dist.:  3.466  2.063  4.562  3.942  7.835 Å

Site 3: 5 lowest local moments
Fe index:     106     22     11     59     47
Mag. mom. : -3.688 -3.781 -3.793 -3.860 -4.012 µB
µ-Fe dist.:  2.055  6.902  6.773  4.562  4.014 Å

For reference, the mean of 10 the largest local moments is
mags:  4.028 µB


For Site 1: one polaron lives on Fe ions 11 and 22 and the other on Fe ions 46 and 95.   
For Site 3: one polaron lives on Fe ions 11 and 22 and the other on Fe ions 106 and 59.   


Interestingly, the second polaron localised on the same spin sublattice as the first. Further exploration, forcing alternative configurations would be interesting to pursue.

#### Summary

In [54]:
# For reference, let us print out the properties of the unseparated µ-polaron complex in C1:
muminus_C1_energy = muminus_C1.get_potential_energy() # eV
mags = abs(muminus_C1.get_magnetic_moments())
muminus_C1_polaron_order =np.argsort(mags[:108])
dist1 = muzero_C1.get_distance(-1, muminus_C1_polaron_order[0], mic=True)
dist2 = muzero_C1.get_distance(-1, muminus_C1_polaron_order[1], mic=True)
print(f"Site: 0, \
Dist µ-pol1: {dist1:5.2f} Å, \
Dist µ-pol2: {dist2:5.2f} Å, \
E wrt C1: {0:5.1f} meV")


print("\nVec a")
for i, atoms in enumerate(vec_a_structures):
    dist1 = atoms.get_distance(-1, vec_a_order[:,0][i], mic=True)
    dist2 = atoms.get_distance(-1, vec_a_order[:,1][i], mic=True)
    e = atoms.get_potential_energy()
    print(f"Site: {i+1}, \
Dist µ-pol1: {dist1:5.2f} Å, \
Dist µ-pol2: {dist2:5.2f} Å, \
E wrt C1: {(e - muminus_C1_energy)*1000:5.1f} meV ")
    
print("\nVec ab")
for i, atoms in enumerate(vec_ab_structures):
    dist1 = atoms.get_distance(-1, vec_ab_order[:,0][i], mic=True)
    dist2 = atoms.get_distance(-1, vec_ab_order[:,1][i], mic=True)
    e = atoms.get_potential_energy()
    print(f"Site: {i+1}, \
Dist µ-pol1: {dist1:5.2f} Å, \
Dist µ-pol2: {dist2:5.2f} Å, \
E wrt C1: {(e - muminus_C1_energy)*1000:5.1f} meV ")

Site: 0, Dist µ-pol1:  2.24 Å, Dist µ-pol2:  2.11 Å, E wrt C1:   0.0 meV

Vec a
Site: 1, Dist µ-pol1:  3.81 Å, Dist µ-pol2:  2.06 Å, E wrt C1: 173.7 meV 
Site: 2, Dist µ-pol1:  2.13 Å, Dist µ-pol2:  2.47 Å, E wrt C1:  -0.0 meV 
Site: 3, Dist µ-pol1:  2.06 Å, Dist µ-pol2:  9.95 Å, E wrt C1: 170.0 meV 

Vec ab
Site: 1, Dist µ-pol1:  3.47 Å, Dist µ-pol2:  2.06 Å, E wrt C1: 161.9 meV 
Site: 2, Dist µ-pol1:  2.05 Å, Dist µ-pol2:  6.90 Å, E wrt C1: 182.2 meV 


For Vec a:   
In the case of Site 2, the second polaron localised right next to the new muon position in a similar configuration to the initial Site 0 (i.e. C$^-_1$) and thus had similar properties.

Sites 1 and 3, however, did successfully trap the polaron further away from the muon resulting in a significant increase in total energy.


For Vec ab:   
Sites 1,2 and 3 successfully trapped the polaron further away from the muon, resulting in a significant increase in total energy. 

We can therefore say that the muon-(two-polaron) complex is more stable than the configurations tested for a muon-polaron complex and separate second polaron.


**Overall we generally see an increase in energy as a function of muon-polaron separation.**